# Neural Machine Translation with Sequence to Sequence Models


In [0]:
__instructor__ = "Dragomir Radev"
__authors__ = "Online Tutorial Author (Link will be revealed later)", "Alexander R. Fabbri", "Tao Yu"
__version__ = "Introduction to Natural Language Processing, Spring 2019"

## Content

0. Overview
0. Environment Setup
0. Assignment Part 1: Vanilla Sequence-to-Sequence Model
0. Assignment Part 2: Sequence-to-Sequence with Attention (in a separate notebook)
0. Assignment Part 3: Transformer Model (Bonus -- in a separate notebook)
0. Submission

## Overview

In this assignment, You will:

* Learn to use some helpful tools such as Google Colab, Pytorch, Torchtext
* Develop a neural model for machine translation


__Sections of this document__

Section A: Background

1. Environment setup
2. Introduction to Sequence-to-Sequence Models

Section B: Implementation of Deep Learning for Machine Translation

3. Implementing a vanilla Sequence-to-Sequence Model
4. Implementing a Sequence-to-Sequence Model with Attention
5. Transformer Model (bonus)

__Grading Criteria__

*  Preprocessing with Torchtext  - ___15 points___
    0. Implement Torchtext preprocessing - ___10 points___
    0. Miscellaneous questions  - ___5 points___
    
* Implement a vanilla sequence-to-sequence model in PyTorch - ___25 points___
    0. Implement encoder – __10__ points
    0. Implement decoder – __10__ points
    0. Train network – __5__ points
    
* Implement a sequence-to-sequence model with attention - ___60 points___
    0. Implement attention mechanism – __35__ points
    0. Modify decoder to include attention – __15__ points
    0. Experiment with code for inference and attention weights - __5__ points
    0. Train network - __5__ points
    
* (Bonus) Implement the Transformer Model - ___10 points___
    
 __Quick Search Tags__
 
* **TODO**: to help you find questions and code snippets you have to address. Please do **NOT** delete this as the TAs will use this tag to quickly find your solutions during grading. 

## Environment Setup

For this assignment, we will use Google Colab. It is a free cloud service based on Jupyter Notebooks that lets you use a free GPU. Basically, you can quickly create, upload, share, and even edit togther Jupyter notebooks.

Please refer to [tutorial 1](https://course.fast.ai/start_colab.html) or [tutorial 2](https://towardsdatascience.com/getting-started-with-google-colab-f2fff97f594c) for additional instructions on how to use Google Colab.

#### Using GPU in Colab
Before running anything, you need to tell Colab that you want to use a GPU: 
1. Go to __Runtime__ option on the top left
2. Click __Change runtime type__
3. Select "Python 3" for __Runtime type__ and "GPU" for __Hardward accelerator__
4. Click __SAVE__ button

Colab has popular libraries already installed such as Pytorch, TensorFlow, OpenCV and Keras. Let's get started and verify this:

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Pytorch version is: ", torch.__version__)
print("You are using: ", device)

Pytorch version is:  1.0.1.post2
You are using:  cuda


You should be using CUDA with Pytorch 1.0.1.post2.

**Note**: Of course, there are some limits. For example, it allows you access to a free GPU for only 10 hours at one time.


 **Note**: You need to finish this assignment on Google Colab  because you will share your Google Colab notebook with TAs for grading.


## Assignment Part 1: Vanilla Sequence to Sequence Model

Cześć! Hello! Bonjour! In this assignment we are going to build a machine learning model for translating from one language to another using PyTorch. We will be working with German to English translation, but the methods and implementation generalize to any language for which there exists an equivalent tokenizer. Also, the sequence-to-sequence models which will be introduced are used in tasks beyond translation, including summarization, dialogue systems, among others. 

## Introduction
Sequence-to-Sequence (seq2seq) models were first introduced in the [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) paper. These models do exactly what you'd think they do; they take one sequence and output another sequence (see [this link](http://jalammar.github.io/images/seq2seq_2.mp4)). Seq2seq models are a type of **encoder-decoder** model, in which a  model such as *recurrent neural network* (RNN) *encodes* the input (also called source) sentence into a single vector, often called the context vector. The context vector is a representation of the entire input sentence, and this vector is then *decoded* by a second model, often another RNN, which learns to output the *target* sentence by generating one word at a time. 


<center><img src="https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/master/assets/seq2seq1.png" alt="mlp" align="middle"></center>

This image shows an example translation using recurrent neural networks. The source sentence, "guten morgen" is input into the encoder (green) one word at a time. We also append a *start of sequence* (`<sos>`) and *end of sequence* (`<eos>`) token to the start and end of sentence, respectively, to give the model an idea of when encoding and decoding start and end. At each time-step, the input to the encoder RNN is both the current word, $x_t$, as well as the hidden state from the previous time-step, $h_{t-1}$, and the encoder RNN outputs a new hidden state $h_t$. You can think of the hidden state as a vector representation of the sentence so far. The RNN can be represented as a function of both of $x_t$ and $h_{t-1}$:

$$h_t = \text{EncoderRNN}(x_t, h_{t-1})$$

We're using the term RNN generally here, it could be any recurrent architecture, such as an *LSTM* (Long Short-Term Memory) or a *GRU* (Gated Recurrent Unit). Additionally, as we will later see, there are non-recurrent models which achieve state-of-the-art results for machine translation. 

Here, we have $X = \{x_1, x_2, ..., x_T\}$, where $x_1 = \text{<sos>}, x_2 = \text{guten}$, etc. The initial hidden state, $h_0$, is usually initialized to zeros.

Once the final word, $x_T$, has been passed into the RNN, we use the final hidden state, $h_T$, as the context vector, i.e. $h_T = z$. This is a vector representation of the entire source sentence.

Now that we have our context vector, $z$, we can start decoding it to get the target sentence, "good morning." Again, we append start and end of sequence tokens to the target sentence. At each time-step, the input to the decoder RNN (blue) is the current word, $y_t$, as well as the hidden state from the previous time-step, $s_{t-1}$, where the initial decoder hidden state, $s_0$, is the context vector, $s_0 = z = h_T$, i.e. the initial decoder hidden state is the final encoder hidden state. Thus, similar to the encoder, we can represent the decoder as:

$$s_t = \text{DecoderRNN}(y_t, s_{t-1})$$

In the decoder, we need to go from the hidden state to an actual word, therefore at each time-step we use $s_t$ to predict (by passing it through a `Linear` layer, shown in purple which projects the hidden state onto the output vocabulary space, to which softmax is applied to pick the most likely word) what we think is the next word in the sequence, $\hat{y}_t$. 

$$\hat{y}_t = f(s_t)$$

We always use `<sos>` for the first input to the decoder, $y_1$, but for subsequent inputs, $y_{t>1}$, we will sometimes use the actual, ground truth next word in the sequence, $y_t$ and sometimes use the word predicted by our decoder, $\hat{y}_{t-1}$. This is called *teacher forcing*, and you can read about it more [here](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/).

When training/testing our model, we always know how many words are in our target sentence, so we stop generating words once we hit that many. During inference (i.e. real world usage where we do not have access to the target translations) it is common to keep generating words until the model outputs an `<eos>` token or after a certain amount of words have been generated.

Once we have our predicted target sentence, $\hat{Y} = \{ \hat{y}_1, \hat{y}_2, ..., \hat{y}_T \}$, we compare it against our actual target sentence, $Y = \{ y_1, y_2, ..., y_T \}$, to calculate our loss. We then use this loss to update all of the parameters in our model.



### Preparing Data

We'll be coding up the models in PyTorch and using TorchText to help us do all of the pre-processing required. We'll also be using spaCy to assist in the tokenization of the data. [SpaCy](https://spacy.io/usage/spacy-101) is a very useful tool for paring and iterating over text in the preprocessing phase. 

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import os
import time

# We'll set the random seeds for deterministic results.
SEED = 1

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.deterministic = True


SpaCy has model for each language ("de" for German and "en" for English) which need to be loaded so we can access the tokenizer of each model. You can experiment with the [other available languages](https://spacy.io/usage/models) from spaCy. 


We have to download the models for German and English and then load the models as such:

In [0]:
%%capture
! python -m spacy download en
! python -m spacy download de
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

Next, we create the tokenizer functions. These can be passed to TorchText and will take in the sentence as a string and return the sentence as a list of tokens.

In the original paper, they find it beneficial to reverse the order of the input when feeding it to the model. 

**TODO**: Q1. Write why you think that would help.  (hint -- see the original paper for their explanation.)

Reversing the order of the input helps becaus it introduces many short term dependencies to the dataset. Without reversing, when source sentence and target sentence are concatenated together, the distance between a word in source sentence and its corresponding word in target sentence is far. By reverting however, the average distance between corresponding words in source sentence and target sentence remains unchanged, but the first few words in source sentence are now very close to the first few words in target sentence, thus greatly reducing the problem's minimal time lag.

**TODO**: Q2. Can you think of any additional data augmentation which could be done to improve translation results? 

1. Thesaurus: Replace words and/or phrases by their synonyms.

2. Word embeddings + cosine similarity: Find similar words for replacement. 

In [0]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

TorchText's `Field`s handle how data should be processed. You can read all of the possible arguments [here](https://github.com/pytorch/text/blob/master/torchtext/data/field.py#L61). 

**TODO**: Please create two fields below, one called SRC for the source input data and one called TRG for the target translated data. Initialize each method with tokenize = their respective language tokenizer (remember that German is the source language here), an init_token  "**<sos\>**" and an eos_token "**<eos\>**" and set lowercasing to true. **For the SRC Field only, also include lengths** of the input following preprocessing. This is important for batching the examples later on. 

Please see [the Field documentation](https://torchtext.readthedocs.io/en/latest/data.html#field). For clarification, you should look at the "tokenize," "init_token," "eos_token," "lower," and "include_lengths" parameters. 




In [0]:
# TODO uncomment and fill this in according to the specifications above
BOS_WORD = '<sos>'
EOS_WORD = '<eos>'
SRC = Field(tokenize=tokenize_de, init_token=BOS_WORD, eos_token=EOS_WORD, lower=True, include_lengths=True)
TRG = Field(tokenize=tokenize_en, init_token=BOS_WORD, eos_token=EOS_WORD, lower=True)

Next, we download and load the train, validation and test data. 

The dataset we'll be using is the [Multi30k dataset](https://github.com/multi30k/dataset). This is a dataset with ~30,000 parallel English, German and French sentences, each with ~12 words per sentence. 

`exts` specifies which languages to use as the source and target (source goes first) and `fields` specifies which field to use for the source and target.

In [7]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 866kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 243kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 240kB/s]


We can double check that we've loaded the right number of examples:

In [8]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


We can also print out an example, making sure the source sentence is reversed:

In [9]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


The period is at the beginning of the German (src) sentence, so looks good!

Next, we'll build the *vocabulary* for the source and target languages. The vocabulary is used to associate each unique token with an index (an integer) and this is used to build a one-hot encoding for each token (a vector of all zeros except for the position represented by the index, which is 1). The vocabularies of the source and target languages are distinct.

Using the `min_freq` argument, we only allow tokens that appear at least 2 times to appear in our vocabulary. Tokens that appear only once are converted into an `<unk>` (unknown) token.

It is important to note that your vocabulary should only be built from the training set and not the validation/test set. This prevents "information leakage" into your model, giving you artifically inflated validation/test scores.

In [0]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [11]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


The final step of preparing the data is to create the iterators. These can be iterated on to return a batch of data which will have a `src` attribute (the PyTorch tensors containing a batch of numericalized source sentences) and a `trg` attribute (the PyTorch tensors containing a batch of numericalized target sentences). Numericalized is just a fancy way of saying they have been converted from a sequence of readable tokens to a sequence of corresponding indexes, using the vocabulary. 

We also need to define a `torch.device`. This is used to tell TorchText to put the tensors on the GPU or not. We use the `torch.cuda.is_available()` function, which will return `True` if a GPU is detected on our computer. We pass this `device` to the iterator.

When we get a batch of examples using an iterator we need to make sure that all of the source sentences are padded to the same length, the same with the target sentences. Luckily, TorchText iterators handle this for us! 

We use a `BucketIterator` instead of the standard `Iterator` as it creates batches in such a way that it minimizes the amount of padding in both the source and target sentences. 

**TODO**: please add a parameter to BucketIterator to sort items within a batch. See [the documentation](https://torchtext.readthedocs.io/en/latest/data.html#bucketiterator) for a list of available parameters. 

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
BATCH_SIZE = 128
# TODO uncomment and modify this
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort_key = lambda x: len(x.src), sort_within_batch=True)

### Building the Seq2Seq Model

We'll be building our model in three parts. The encoder, the decoder and a seq2seq model that encapsulates the encoder and decoder and will provide a way to interface with each.

### Encoder

First, the encoder, a single layer GRU. The paper we are implementing uses a 4-layer LSTM, but in the interest of training time and simplicity we cut this down to a single layer GRU. The concept of multi-layer RNNs is easy to expand from a single layer to 4 layers. See [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) blog post if you want to learn more about GRUs and LSTMs. You just need to know that they're a type of RNN. 

$$\begin{align*}
h_t &= \text{RNN}(x_t, h_{t-1})\\
h_t &= \text{GRU}(x_t, h_{t-1})\\
\end{align*}$$


We create this in code by making an `Encoder` module, which requires we inherit from `torch.nn.Module` and use the `super().__init__()` as some boilerplate code. The encoder takes the following arguments:
- `input_dim` is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
- `emb_dim` is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with `emb_dim` dimensions. 
- `enc_hid_dim` is the dimensionality of the hidden states in the encoder.
- `dec_hid_dim` is the dimensionality of the hidden states in the decoder, which may differ from the hid_dim of the encoder.
- `dropout` is the amount of dropout to use. This is a regularization parameter to prevent overfitting. Check out [this](https://www.coursera.org/lecture/deep-neural-network/understanding-dropout-YaGbR) for more details about dropout.

If you would like to know about word embeddings, I'd recommend these articles [1](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/), [2](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html), [3](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), [4](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/). 

The embedding layer is created using `nn.Embedding`, the GRU with `nn.GRU` and a dropout layer with `nn.Dropout`. Check the PyTorch [documentation](https://pytorch.org/docs/stable/nn.html) for more about these.

One thing to note is that the `dropout` argument to the GRU is how much dropout to apply between the layers of a multi-layer RNN, i.e. between the hidden states output from layer $l$ and those same hidden states being used for the input of layer $l+1$.

In the `forward` method, we pass in the source sentence, $X$, which is converted into dense vectors using the `embedding` layer, and then dropout is applied. These embeddings are then passed into the RNN. As we pass a whole sequence to the RNN, it will automatically do the recurrent calculation of the hidden states over the whole sequence for us! You may notice that we do not pass an initial hidden state to the RNN. This is because, as noted in the [documentation](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU), that if no hidden state is passed to the RNN, it will automatically create an initial hidden state as a tensor of all zeros. 

The RNN returns: `outputs` (the top-layer hidden state for each time-step) and `hidden` (the final hidden state for each layer, $h_T$, stacked on top of each other).


The sizes of each of the tensors is left as comments in the code.

**TODO** 
1. in the \__init__ function below, fill in complete self.rnn with a [GRU](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU) of (emb_dim input dimension, enc_hid_dim output dimension) and which is bidirectional. Also, create a linear layer called self.fc which has input dimension enc_hid_dim * 2 and output dimension dec_hid_dim. 

2. Fill in the code in the forward function with the appropriate function for padding and unpacking sequences of varying length. You just have to read the documentation for [pack_padded_sequence ](https://pytorch.org/docs/stable/nn.html#pack-padded-sequence) and [pad_padded_sequence](https://pytorch.org/docs/stable/nn.html#pad-packed-sequence) and fill in the code with the appropriate function. 

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # TODO uncomment and fill in 
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        # TODO uncomment and fill in 
        self.fc = nn.Linear(2 * enc_hid_dim, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        # src = [src sent len, batch size]
        # src_len = [src sent len]
  
        embedded = self.dropout(self.embedding(src))
        # embedded = [src sent len, batch size, emb dim]
      
        # TODO uncomment and fill in with correct padded sequence function
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
        
        packed_outputs, hidden = self.rnn(packed_embedded)
        # TODO uncomment and fill in with correct padded sequence function               
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        # outputs = [sent len, batch size, dec_hid_dim * num directions]
        # hidden = [n layers * num directions, batch size, dec_hid_dim]
        
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        
        # hidden [-2, :, : ] is the last of the forwards RNN 
        # hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        
        # outputs = [sent len, batch size, enc hid dim * 2]
        # hidden = [batch size, dec_hid_dim]
        
        return outputs, hidden

### Decoder

Next, we'll build our decoder, which will be a 1-layer (4-layer LSTM in the paper) GRU.

![](assets/seq2seq3.png)

The `Decoder` class does a single step of decoding. In this example, we are using a single layer GRU, so the equation will look just like the encoder equation: 



$$\begin{align*}
s_t = \text{DecoderGRU}(y_t, s_{t-1})\\
\end{align*}$$


Remember that the initial hidden state to our decoder are our context vectors, which are the final hidden state of our encoder, i.e. $s_0=z=h_T$.

We then pass the hidden state from the RNN, $s_t$, through a linear layer, $f$, to make a prediction of what the next token in the target (output) sequence should be, $\hat{y}_{t+1}$. 

$$\hat{y}_{t+1} = f(s_t)$$

The arguments and initialization are analogous to the `Encoder` class, except we now have an `output_dim` which is the size of the one-hot vectors that will be input to the decoder. These are equal to the vocabulary size of the output/target. There is also the addition of the `Linear` layer, used to make the predictions from the top layer hidden state.

Within the `forward` method, we accept a batch of input tokens, previous hidden states. We `unsqueeze` the input tokens to add a sentence length dimension of 1. Then, similar to the encoder, we pass through an embedding layer. This batch of embedded tokens is then passed into the RNN with the previous hidden state. This produces an `output` (hidden state from the top layer of the RNN), a new `hidden` state. We then pass the `output` (after getting rid of the sentence length dimension) through the linear layer to receive our `prediction`. We then return the `prediction` and the new `hidden` state. 

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers)
        
        self.out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, input, hidden):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        # input = [1, batch size]
        
        embedded = self.embedding(input)
        
        # embedded = [1, batch size, emb dim]
        
        output, hidden = self.rnn(embedded, hidden)
        
        # output = [sent len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        # sent len and n directions will always be 1 in the decoder, therefore:
        # output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        prediction = self.out(output.squeeze(0))
        
        # prediction = [batch size, output dim]
        
        return prediction, hidden

### Seq2Seq

For the final part of the implementation, we'll implement the seq2seq model. This will handle: 
- receiving the input/source sentence
- using the encoder to produce the context vectors 
- using the decoder to produce the predicted output/target sentence


The `Seq2Seq` model takes in an `Encoder`, `Decoder`, and a `device` (used to place tensors on the GPU, if it exists).

Our `forward` method takes the source sentence, target sentence and a teacher-forcing ratio. The teacher forcing ratio is used when training our model. When decoding, at each time-step we will predict what the next token in the target sequence will be from the previous tokens decoded, $\hat{y}_{t+1}=f(s_t)$. With probability equal to the teaching forcing ratio (`teacher_forcing_ratio`) we will use the actual ground-truth next token in the sequence as the input to the decoder during the next time-step. However, with probability `1 - teacher_forcing_ratio`, we will use the token that the model predicted as the next input to the model, even if it doesn't match the actual next token in the sequence.  

The first thing we do in the `forward` method is to create an `outputs` tensor that will store all of our predictions, $\hat{Y}$.

We then feed the input/source sentence, $X$/`src`, into the encoder and receive out final hidden and cell states.

The first input to the decoder is the start of sequence (`<sos>`) token. As our `trg` tensor already has the `<sos>` token appended (all the way back when we defined the `init_token` in our `TRG` field) we get our $y_1$ by slicing into it. We know how long our target sentences should be (`max_len`), so we loop that many times. During each iteration of the loop, we:
- pass the input, previous hidden states ($y_t, s_{t-1}$) into the decoder
- receive a prediction, next hidden state ($\hat{y}_{t+1}, s_{t}$) from the decoder
- place our prediction, $\hat{y}_{t+1}$/`output` in our tensor of predictions, $\hat{Y}$/`outputs`
- decide if we are going to "teacher force" or not
    - if we do, the next `input` is the ground-truth next token in the sequence, $y_{t+1}$/`trg[t]`
    - if we don't, the next `input` is the predicted next token in the sequence, $\hat{y}_{t+1}$/`top1`
    
Once we've made all of our predictions, we return our tensor full of predictions, $\hat{Y}$/`outputs`.

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.enc_hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        # src = [src sent len, batch size]
        # trg = [trg sent len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        _, hidden = self.encoder(src, src_len)
        hidden = hidden.unsqueeze(0)
        # first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):  
            output, hidden = self.decoder(input, hidden)
            outputs[t] = output
            # choose whether to do teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs

### Training the Seq2Seq Model

Now we have our model implemented, we can begin training it. 

First, we'll initialize our model. As mentioned before, the input and output dimensions are defined by the size of the vocabulary. The embedding dimesions and dropout for the encoder and decoder can be different, but the number of layers and the size of the hidden/cell states must be the same. We use a single layer in these experiments. 

We then define the encoder, decoder and then our Seq2Seq model, which we place on the `device`.

In [0]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS)

model = Seq2Seq(enc, dec, device).to(device)

We also define a function that will calculate the number of trainable parameters in the model.

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,615,045 trainable parameters


We define our optimizer, which we use to update our parameters in the training loop. Check out [this](http://ruder.io/optimizing-gradient-descent/) post for information about different optimizers. Here, we'll use Adam.

In [0]:
optimizer = optim.Adam(model.parameters())

Next, we define our loss function. The `CrossEntropyLoss` function calculates both the log softmax as well as the negative log-likelihood of our predictions. 

Our loss function calculates the average loss per token, however by passing the index of the `<pad>` token as the `ignore_index` argument we ignore the loss whenever the target token is a padding token. 

In [0]:
PAD_IDX = TRG.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

Next, we'll define our training loop. 

First, we'll set the model into "training mode" with `model.train()`. This will turn on dropout (and batch normalization, which we aren't using) and then iterate through our data iterator.

At each iteration:
- get the source and target sentences from the batch, $X$ and $Y$
- zero the gradients calculated from the last batch
- feed the source and target into the model to get the output, $\hat{Y}$
- as the loss function only works on 2d inputs with 1d targets we need to flatten each of them with `.view`
    - we also don't want to measure the loss of the `<sos>` token, hence we slice off the first column of the output and target tensors
- calculate the gradients with `loss.backward()`
- clip the gradients to prevent them from exploding (a common issue in RNNs)
- update the parameters of our model by doing an optimizer step
- sum the loss value to a running total

Finally, we return the loss that is averaged over all batches.

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src, src_len = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()

        output = model(src, src_len, trg)
        
        # trg = [trg sent len, batch size]
        # output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        # trg = [(trg sent len - 1) * batch size]
        # output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Our evaluation loop is similar to our training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

We must remember to set the model to evaluation mode with `model.eval()`. This will turn off dropout (and batch normalization, if used).

We use the `with torch.no_grad()` block to ensure no gradients are calculated within the block. This reduces memory consumption and speeds things up. 

The iteration loop is similar (without the parameter updates), however we must ensure we turn teacher forcing off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment.

In [0]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0) # turn off teacher forcing

            # trg = [trg sent len, batch size]
            # output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            # trg = [(trg sent len - 1) * batch size]
            # output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Next, we'll create a function that we'll use to tell us how long an epoch takes.

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We can finally start training our model!

At each epoch, we'll be checking if our model has achieved the best validation loss so far. If it has, we'll update our best validation loss and save the parameters of our model (called `state_dict` in PyTorch). Then, when we come to test our model, we'll use the saved parameters used to achieve the best validation loss. 

We'll be printing out both the loss and the perplexity at each epoch. It is easier to see a change in perplexity than a change in loss as the numbers are much bigger.

**TODO** Train the model for 5 epochs and report your results (perplexity). 
I was getting results like this:

Epoch: 01 | Time: 0m 42s

	Train Loss: 4.740 | Train PPL: 114.394
	 Val. Loss: 4.912 |  Val. PPL: 135.867
   
   Epoch: 02 | Time: 0m 42s
   
	Train Loss: 3.924 | Train PPL:  50.603
	 Val. Loss: 4.226 |  Val. PPL:  68.411

In [24]:
N_EPOCHS = 5
CLIP = 1
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'tut1_model.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 22s
	Train Loss: 4.744 | Train PPL: 114.904
	 Val. Loss: 4.944 |  Val. PPL: 140.333
Epoch: 02 | Time: 0m 22s
	Train Loss: 3.935 | Train PPL:  51.143
	 Val. Loss: 4.254 |  Val. PPL:  70.382
Epoch: 03 | Time: 0m 22s
	Train Loss: 3.436 | Train PPL:  31.050
	 Val. Loss: 4.059 |  Val. PPL:  57.916
Epoch: 04 | Time: 0m 22s
	Train Loss: 3.154 | Train PPL:  23.423
	 Val. Loss: 3.830 |  Val. PPL:  46.042
Epoch: 05 | Time: 0m 22s
	Train Loss: 2.908 | Train PPL:  18.324
	 Val. Loss: 3.737 |  Val. PPL:  41.992


We'll load the parameters (`state_dict`) that gave our model the best validation loss and run it the model on the test set.

In [25]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.741 | Test PPL:  42.146 |


## Submission

Now that you have completed the assignment, follow the steps below to submit your aissgnment:
1. Click __Runtime__  > __Run all__ to generate the output for all cells in the notebook.
2. Save the notebook with the output from all the cells in the notebook by click __File__ > __Download .ipynb__.
3. Copy model train and test prints, answers to all short questions, and the shareable link of this notebook to a `README.txt` file.
4. Put the .ipynb file and `README.txt` under your hidden directory on the Zoo server `~/hidden/<YOUR_PIN>/Homework5/`.
5. As a final step, run a script that will set up the permissions to your homework files, so we can access and run your code to grade it. Make sure the command runs without errors, and do not make any changes or run the code again. If you do run the code again or make any changes, you need to run the permissions script again. Submissions without the correct permissions may incur some grading penalty.
`/home/classes/cs477/bash_files/hw5_set_permissions.sh <YOUR_PIN>`